In [4]:
import pygame
import random
import math

pygame.init()

# Constants
WINDOW_SIZE = (600, 600)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
LIGHT_GREEN = (144, 238, 144)  # For path prediction
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 3
tolerance = 10
ARROW_LENGTH = 20
NUM_GOALS = 3
PATH_POINTS = 20  # Number of points to show in predicted path

# Increased step size for agent1 (green dot)
agent1_step_size = 3.5  # Doubled from the original step size

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Path Prediction")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Agent positions and directions
agent1_pos = [300, 300]
agent2_pos = [300, 300]
agent3_pos = [300, 300]
agent1_dir = [0, 0]
agent2_dir = [0, 0]
agent3_dir = [0, 0]
reached_goal = False

# Create multiple targets
targets = []
for _ in range(NUM_GOALS):
    targets.append([random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])])

current_target_idx = 0
prev_human_pos = agent2_pos.copy()

def calculate_path_points(start_pos, target_pos, num_points=PATH_POINTS):
    """Calculate points along the predicted path"""
    points = []
    dx = target_pos[0] - start_pos[0]
    dy = target_pos[1] - start_pos[1]
    distance = math.sqrt(dx**2 + dy**2)
    
    if distance == 0:
        return points
        
    # Calculate step sizes
    step_x = dx / (num_points - 1)
    step_y = dy / (num_points - 1)
    
    for i in range(num_points):
        x = start_pos[0] + step_x * i
        y = start_pos[1] + step_y * i
        points.append((int(x), int(y)))
    
    return points

def draw_predicted_path(surface, points, color):
    """Draw a dotted line showing the predicted path"""
    if len(points) < 2:
        return
        
    # Draw dots with decreasing size and opacity
    for i in range(len(points) - 1):
        # Calculate dot size and opacity
        size = max(1, int(5 * (1 - i/len(points))))
        alpha = int(255 * (1 - i/len(points)))
        
        # Create a surface for the dot with transparency
        dot_surface = pygame.Surface((size*2, size*2), pygame.SRCALPHA)
        color_with_alpha = (*color[:3], alpha)
        pygame.draw.circle(dot_surface, color_with_alpha, (size, size), size)
        
        # Blit the dot
        surface.blit(dot_surface, (points[i][0] - size, points[i][1] - size))

def distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def predict_human_target():
    global current_target_idx, prev_human_pos
    
    human_dx = agent2_pos[0] - prev_human_pos[0]
    human_dy = agent2_pos[1] - prev_human_pos[1]
    
    if human_dx == 0 and human_dy == 0:
        return current_target_idx
    
    best_score = float('-inf')
    best_target_idx = current_target_idx
    
    for i, target in enumerate(targets):
        to_target_dx = target[0] - agent2_pos[0]
        to_target_dy = target[1] - agent2_pos[1]
        
        movement_mag = math.sqrt(human_dx**2 + human_dy**2)
        target_mag = math.sqrt(to_target_dx**2 + to_target_dy**2)
        
        if movement_mag == 0 or target_mag == 0:
            continue
            
        alignment = (human_dx * to_target_dx + human_dy * to_target_dy) / (movement_mag * target_mag)
        
        dist = distance(agent2_pos, target)
        max_dist = math.sqrt(WINDOW_SIZE[0]**2 + WINDOW_SIZE[1]**2)
        distance_factor = 1 - (dist / max_dist)
        
        score = (alignment * 0.7) + (distance_factor * 0.3)
        
        if score > best_score:
            best_score = score
            best_target_idx = i
            
        
    if best_target_idx != current_target_idx:
        agent1_pos[0] = agent2_pos[0]
        agent1_pos[1] = agent2_pos[1]
    
    return best_target_idx

def calculate_gamma():
    max_distance = math.sqrt(WINDOW_SIZE[0]**2 + WINDOW_SIZE[1]**2)
    distance_w = distance(agent1_pos, targets[current_target_idx])
    distance_h = distance(agent2_pos, targets[current_target_idx])
    
    # Normalize distances
    normalized_distance_w = distance_w / max_distance
    normalized_distance_h = distance_h / max_distance
    
    if normalized_distance_w + normalized_distance_h == 0:
        return 0.5
    else:
        gamma = normalized_distance_h / (normalized_distance_w + normalized_distance_h)
        
    gamma = gamma ** 2
    return gamma

def normalize_vector(dx, dy):
    length = math.sqrt(dx**2 + dy**2)
    if length == 0:
        return 0, 0
    return dx/length, dy/length

def draw_arrow(surface, color, start_pos, direction, length=ARROW_LENGTH):
    if direction[0] == 0 and direction[1] == 0:
        return
        
    end_x = start_pos[0] + direction[0] * length
    end_y = start_pos[1] + direction[1] * length
    
    pygame.draw.line(surface, color, start_pos, (end_x, end_y), 2)
    
    arrow_size = 7
    angle = math.atan2(direction[1], direction[0])
    arrow1_x = end_x - arrow_size * math.cos(angle + math.pi/6)
    arrow1_y = end_y - arrow_size * math.sin(angle + math.pi/6)
    arrow2_x = end_x - arrow_size * math.cos(angle - math.pi/6)
    arrow2_y = end_y - arrow_size * math.sin(angle - math.pi/6)
    
    pygame.draw.line(surface, color, (end_x, end_y), (arrow1_x, arrow1_y), 2)
    pygame.draw.line(surface, color, (end_x, end_y), (arrow2_x, arrow2_y), 2)

def move_agent1():
    global agent1_pos, reached_goal, agent1_dir, current_target_idx
    current_target = targets[current_target_idx]
    dx = current_target[0] - agent1_pos[0]
    dy = current_target[1] - agent1_pos[1]
    distance_to_target = math.sqrt(dx**2 + dy**2)
    
    agent1_dir = normalize_vector(dx, dy)
    
    if distance_to_target > agent1_step_size:
        move_ratio = agent1_step_size / distance_to_target
        agent1_pos[0] += dx * move_ratio
        agent1_pos[1] += dy * move_ratio
    else:
        agent1_pos[0] = current_target[0]
        agent1_pos[1] = current_target[1]
        
        # Check if we've reached the last target
        if current_target_idx == len(targets) - 1:
            reached_goal = True
            # Auto-reset after a short delay
            pygame.time.set_timer(pygame.USEREVENT, 1000)  # 1 second delay
        else:
            # Move to next target
            current_target_idx += 1
            agent1_pos[0] = agent2_pos[0]
            agent1_pos[1] = agent2_pos[1]   

def move_agent2(dx, dy):
    global agent2_pos, agent2_dir, prev_human_pos
    prev_human_pos = agent2_pos.copy()
    
    new_x = agent2_pos[0] + dx
    new_y = agent2_pos[1] + dy
    agent2_pos[0] = max(0, min(WINDOW_SIZE[0], new_x))
    agent2_pos[1] = max(0, min(WINDOW_SIZE[1], new_y))
    
    if dx != 0 or dy != 0:
        agent2_dir = normalize_vector(dx, dy)

def update_agent3():
    global agent3_pos, agent3_dir
    gamma = calculate_gamma()
    new_x = gamma * agent1_pos[0] + (1 - gamma) * agent2_pos[0]
    new_y = gamma * agent1_pos[1] + (1 - gamma) * agent2_pos[1]
    
    dx = new_x - agent3_pos[0]
    dy = new_y - agent3_pos[1]
    agent3_dir = normalize_vector(dx, dy)
    
    agent3_pos[0] = new_x
    agent3_pos[1] = new_y
    return gamma

def reset():
    global agent1_pos, agent2_pos, agent3_pos, reached_goal, targets
    global agent1_dir, agent2_dir, agent3_dir, current_target_idx, prev_human_pos
    
    agent1_pos = [300, 300]
    agent2_pos = [300, 300]
    agent3_pos = [300, 300]
    agent1_dir = [0, 0]
    agent2_dir = [0, 0]
    agent3_dir = [0, 0]
    reached_goal = False
    prev_human_pos = agent2_pos.copy()
    current_target_idx = 0
    
    # Reset targets
    targets = []
    for _ in range(NUM_GOALS):
        targets.append([random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])])
        
    # Clear any pending auto-reset events
    pygame.time.set_timer(pygame.USEREVENT, 0)

def render(gamma):
    screen.fill(WHITE)
    
    # Draw predicted path for green dot
    current_target = targets[current_target_idx]
    path_points = calculate_path_points(agent1_pos, current_target)
    draw_predicted_path(screen, path_points, (*GREEN, 128))  # Semi-transparent green
    
    # Draw all targets with numbers
    for i, target in enumerate(targets):
        pygame.draw.circle(screen, YELLOW, (int(target[0]), int(target[1])), 10)
        num_text = font.render(str(i+1), True, BLACK)
        screen.blit(num_text, (target[0] - 5, target[1] - 12))
    
    # Highlight current target with a ring
    pygame.draw.circle(screen, BLACK, (int(current_target[0]), int(current_target[1])), 12, 2)
    
    # Draw agents and their direction arrows
    pygame.draw.circle(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), 10)
    pygame.draw.circle(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), 10)
    pygame.draw.circle(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), 10)
    
    draw_arrow(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), agent1_dir)
    draw_arrow(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), agent2_dir)
    draw_arrow(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), agent3_dir)

    # Draw labels
    screen.blit(font.render("W", True, BLACK), (agent1_pos[0] - 5, agent1_pos[1] - 12))
    screen.blit(font.render("H", True, BLACK), (agent2_pos[0] - 5, agent2_pos[1] - 12))
    screen.blit(font.render("X", True, BLACK), (agent3_pos[0] - 5, agent3_pos[1] - 12))

    # Display information
    gamma_text = font.render(f"Gamma: {gamma:.2f}", True, FONT_COLOR)
    screen.blit(gamma_text, (10, 10))
    formula_text = font.render(f"X = {gamma:.2f}W + {1-gamma:.2f}H", True, FONT_COLOR)
    screen.blit(formula_text, (10, 40))
    target_text = font.render(f"Current Target: {current_target_idx + 1}", True, FONT_COLOR)
    screen.blit(target_text, (10, 70))

    if reached_goal:
        reset_text = font.render("Goal Reached! Auto-resetting...", True, FONT_COLOR)
        screen.blit(reset_text, (150, 100))

    pygame.display.update()

# Main game loop
running = True
clock = pygame.time.Clock()
gamma = 0.5

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                reset()
        if event.type == pygame.USEREVENT:
            reset()  # Auto-reset when timer expires

    if not reached_goal:
        keys = pygame.key.get_pressed()
        dx, dy = 0, 0
        if keys[pygame.K_LEFT]:
            dx -= step_size
        if keys[pygame.K_RIGHT]:
            dx += step_size
        if keys[pygame.K_UP]:
            dy -= step_size
        if keys[pygame.K_DOWN]:
            dy += step_size

        if dx != 0 or dy != 0:
            move_agent2(dx, dy)
            current_target_idx = predict_human_target()
            move_agent1()
            gamma = update_agent3()

    render(gamma)
    clock.tick(30)

pygame.quit()